<a href="https://colab.research.google.com/github/ejrtks1020/Computer_Vision_Project/blob/main/Kaggle-Object-Detection/Kaggle_Chess_Piece_Detection_Train_(Yolo_v5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chess Piece Object Detection Dataset. 
Labeled with LabelImg.

https://www.kaggle.com/tannergi/chess-piece-detection

#1. Ultralytics Yolov5 설치

In [47]:
# Clone and install YOLOv5
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
%cd ..

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
/content


#2. 데이터 다운로드

In [48]:
import os
os.environ['KAGGLE_USERNAME'] = 'asdsadwqrr'
os.environ['KAGGLE_KEY'] = 'fcd162570bf9e9f427cde4418405b341'

In [49]:
!kaggle datasets download -d tannergi/chess-piece-detection

chess-piece-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip '*.zip'

In [51]:
# Ultralytics Yolo images와 labels 디렉토리를 train, val 용으로 생성
!mkdir /content/Chess;
!mkdir /content/Chess/images; mkdir /content/Chess/labels
!mkdir /content/Chess/images/train; mkdir /content/Chess/images/val
!mkdir /content/Chess/labels/train; mkdir /content/Chess/labels/val

mkdir: cannot create directory ‘/content/Chess’: File exists
mkdir: cannot create directory ‘/content/Chess/images’: File exists
mkdir: cannot create directory ‘/content/Chess/labels’: File exists
mkdir: cannot create directory ‘/content/Chess/images/train’: File exists
mkdir: cannot create directory ‘/content/Chess/images/val’: File exists
mkdir: cannot create directory ‘/content/Chess/labels/train’: File exists
mkdir: cannot create directory ‘/content/Chess/labels/val’: File exists


# 3. 데이터 셋의 annotation을 Ultralytics Yolo format으로 생성
* annotation용 xml 파일을 txt 파일로 변환
* 하나의 이미지는 하나의 txt 파일로 변환
* 확장자를 제외한 이미지의 파일명과 annotation 파일명이 서로 동일해야 함.
* 하나의 xml annotation 파일을 Yolo 포맷용 txt 파일로 변환하는 함수 생성
* voc annotation의 좌상단(Top left: x1, y1), 우하단(Bottom right: x2, y2) 좌표를 Bounding Box 중심 좌표(Center_x, Center_y)와 너비(width), 높이(height)로 변경
* 중심 좌표와 너비, 높이는 원본 이미지 레벨로 scale 되어야 함. 모든 값은 0~1 사이 값으로 변환됨.
* class_id는 여러개의 label들을 0 부터 순차적으로 1씩 증가시켜 id 부여

In [52]:
!cat "/content/Chess Detection/annotations/IMG_1991.xml"

<annotation>
	<folder>Chess Detection</folder>
	<filename>IMG_1991.JPG</filename>
	<path>C:\Users\Gilbert\Google Drive\Brand\Datasets\Chess Detection\IMG_1991.JPG</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>4000</width>
		<height>2248</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>white-rook</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>1325</xmin>
			<ymin>1112</ymin>
			<xmax>1564</xmax>
			<ymax>1378</ymax>
		</bndbox>
	</object>
</annotation>


In [53]:
import glob
import xml.etree.ElementTree as ET
import cv2
xml_file = '/content/Chess Detection/annotations/IMG_1989.xml'
tree = ET.parse(xml_file)
root = tree.getroot()

In [54]:
img_node = root.find('size')
img_node

<Element 'size' at 0x7f83a9d4bcb0>

In [55]:
img_node.find('width').text, img_node.find('height').text

('4000', '2248')

In [56]:
w, h = int(img_node.find('width').text), int(img_node.find('height').text)
# 만약 annot값이 0이거나 없다면 이미지를 읽어서 width height 값을 구한다
if w == '' or w == 0 or h =='' or h == 0:
  img_path = xml_file.replace('annotations', 'images').replace('.xml', '.JPG')
  w, h = cv2.imread(img_path).shape[:2]

In [57]:
w, h

(4000, 2248)

In [58]:
obj = root.find('object')
xmlbox = obj.find('bndbox')
x1 = int(xmlbox.find('xmin').text)
y1 = int(xmlbox.find('ymin').text)
x2 = int(xmlbox.find('xmax').text)
y2 = int(xmlbox.find('ymax').text)

(x1, y1), (x2, y2)

((1780, 573), (2319, 1442))

In [59]:
object_name = obj.find('name').text
object_name

'white-rook'

In [60]:
CLASS_NAMES = []

In [61]:
center_x = (x1 + x2) / 2
center_y = (y1 + y2) / 2
bbox_width = x2 - x1
bbox_height = y2 - y1
center_x_norm = round(center_x / w, 7)
center_y_norm = round(center_y / h, 7)
bbox_width_norm = round(bbox_width / w, 7)
bbox_height_norm = round(bbox_height / h, 7)

if object_name not in CLASS_NAMES:
  CLASS_NAMES.append(object_name)
class_id = CLASS_NAMES.index(object_name)

center_x_norm, center_y_norm, bbox_width_norm, bbox_height_norm, class_id

(0.512375, 0.4481762, 0.13475, 0.3865658, 0)

In [64]:
def xml_to_txt(input_xml_file, output_txt_file):
  # ElementTree로 입력 XML파일 파싱
  tree = ET.parse(input_xml_file)
  root = tree.getroot()
  img_node = root.find('size')
  # 만약 img_node를 찾지 못하면 해당 xml파일을 변환 종료
  if img_node is None:
    return None

  # 원본 이미지의 너비와 높이 추출.
  img_width = int(img_node.find('width').text)
  img_height = int(img_node.find('height').text)

  # 만약 너비와 높이 값이 0이라면 이미지 파일을 읽어서 너비와 높이를 구함
  if img_width == 0 or img_height == 0:
    img_height, img_width = cv2.imread(input_xml_file.replace('.xml', '.jpg')).shape[:2]
  
  # xml 파일내에 있는 모든 object Element를 찾는다.
  with open(output_txt_file, 'w') as output_fpointer:
    for obj in root.findall('object'):
      # bndbox를 찾아서 좌상단(x1, y1), 우하단(x2, y2) 좌표 추출.
      xmlbox = obj.find('bndbox')
      x1 = int(xmlbox.find('xmin').text)
      y1 = int(xmlbox.find('ymin').text)
      x2 = int(xmlbox.find('xmax').text)
      y2 = int(xmlbox.find('ymax').text)

      object_name = obj.find('name').text

      # 만약 좌표중에 0보다 작은 값이 있으면 해당 xml파일 변환 종료
      if (x1 < 0) or (y1 < 0) or (x2 < 0) or (y2 < 0):
        break
      
      # object_name 원본 좌표를 입력하여 Yolo 포맷으로 변환하는 함수 호출
      class_id, cx_norm, cy_norm, bbw_norm, bbh_norm = convert_yolo_coord(object_name, img_width, img_height, x1, y1, x2, y2)

      #변환된 yolo 좌표를 object별로 출력 text 파일에 write
      value_str = ('{0} {1} {2} {3} {4}').format(class_id, cx_norm, cy_norm, bbw_norm, bbh_norm)
      output_fpointer.write(value_str + '\n')

def convert_yolo_coord(object_name, img_width, img_height, x1, y1, x2, y2):
  # class_id는 CLASS_NAMES 리스트에서 index 번호로 추출.
  if object_name not in CLASS_NAMES:
    CLASS_NAMES.append(object_name)
  class_id = CLASS_NAMES.index(object_name)
  # 중심 좌표와 너비, 높이 계산. 
  center_x = (x1 + x2)/2
  center_y = (y1 + y2)/2
  width = x2 - x1
  height = y2 - y1
  # 원본 이미지 기준으로 중심 좌표와 너비 높이를 0-1 사이 값으로 scaling
  center_x_norm = center_x / img_width
  center_y_norm = center_y / img_height
  width_norm = width / img_width
  height_norm = height / img_height

  return class_id, round(center_x_norm, 7), round(center_y_norm, 7), round(width_norm, 7), round(height_norm, 7)

In [66]:
xml_to_txt('/content/Chess Detection/annotations/IMG_1989.xml', '/content/Chess/labels/train/IMG_1989.txt')

## Train data, Validation data 분리

In [75]:
from sklearn.model_selection import train_test_split
total_img_paths = glob.glob('/content/Chess Detection/images/*.JPG')
data_id = [path.split('/')[-1][:-4] for path in total_img_paths]
train, val = train_test_split(data_id, test_size = 0.15, random_state = 42)

## VOC Format의 여러개 xml파일들을 Yolo Format으로 변환

In [82]:
import shutil
def make_yolo_anno_file(src_dir, target_dir, data = 'train'):
  src_img_dir = src_dir + '/images/'
  src_annot_dir = src_dir + '/annotations/'

  if data == 'train':
    src_img_paths = [src_img_dir + name + '.JPG' for name in train]
    src_annot_paths = [src_annot_dir + name + '.xml' for name in train]
  else:
    src_img_paths = [src_img_dir + name + '.JPG' for name in val]
    src_annot_paths = [src_annot_dir + name + '.xml' for name in val]

  for i in src_img_paths:
    shutil.copy(i, target_dir + '/images/' + data + '/' + i.split('/')[-1])
  
  for i in src_annot_paths:
    xml_to_txt(i, target_dir + '/labels/' + data + '/' + i.split('/')[-1].replace('.xml', '.txt'))
    
# train용 images와 label annotation 생성
make_yolo_anno_file('/content/Chess Detection', '/content/Chess', 'train')
# val용 images와 label annotation 생성
make_yolo_anno_file('/content/Chess Detection', '/content/Chess', 'val')

In [86]:
print(CLASS_NAMES)

['white-rook', 'black-bishop', 'black-knight', 'white-king', 'black-rook', 'white-pawn', 'white-bishop', 'white-queen', 'black-queen', 'black-king', 'black-pawn', 'white-knight']


#4. Chess Piece Image Dataset 학습 수행

In [84]:
#train log와 weights를 저장할 경로 생성
!mkdir /content/drive/MyDrive/Chess_images

In [88]:
!python yolov5/train.py --img 1024 --batch 8 --epochs 60 \
--data '/content/Chess/chess.yaml' \
--weights yolov5/yolov5s.pt --cache \
--project /content/drive/MyDrive/Chess_images --name chess --exist-ok

train: weights=yolov5/yolov5s.pt, cfg=, data=/content/Chess/chess.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=60, batch_size=8, imgsz=1024, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/Chess_images, name=chess, exist_ok=True, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-266-g07221f1 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.0